# Proyecto Malena

Pasos a aplicar para obtener un modelo predictivo óptimo (top 10) para el 12 de Octubre.

- generar un dataset limpio 
- agregar mis FE 
- agregar deltas, entrenar con otros meses, pasar el index de inflación
- pasar canaritos para eliminar variables que no me sirven 
- Correr un LightGBM model HT - ver que parametros extra se pueden tunear (si quiero parametrizar otras variables hay que mover el dtrain a la funcion de estimar ganancia) 
- Narrow search of LightGBM
- ensemble learning semillerio con mis mejores parametros 


considero: 
http://localhost:8888/edit/Desktop/itbamat2/datasets/exp_FE7130_dataset_7130.csv.gz

Necesito agregar: 
> lags de orden 2 y 3 
> mis fe iniciales
> pasar index de inflación
> pasar canaritos 

In [ ]:
# source( "~/labo/src/FeatureEngineering/z815_FE_final.r")
#Necesita para correr en Google Cloud
# 256 GB de memoria RAM
# 256 GB de espacio en el disco local
#   8 vCPU


#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("Rcpp")

require("ranger")
require("randomForest")  #solo se usa para imputar nulos

require("lightgbm")

options(error = function() { 
  traceback(20); 
  options(error = NULL); 
  stop("exiting after script error") 
})


#------------------------------------------------------------------------------

CorregirCampoMes  <- function( pcampo, pmeses )
{
  tbl <- dataset[  ,  list( "v1" = shift( get(pcampo), 1, type="lag" ),
                            "v2" = shift( get(pcampo), 1, type="lead" )
                         ), 
                   by=numero_de_cliente ]
  
  tbl[ , numero_de_cliente := NULL ]
  tbl[ , promedio := rowMeans( tbl,  na.rm=TRUE ) ]
  
  dataset[ ,
           paste0(pcampo) := ifelse( !(foto_mes %in% pmeses),
                                     get( pcampo),
                                     tbl$promedio ) ]
}
#------------------------------------------------------------------------------
# reemplaza cada variable ROTA  (variable, foto_mes)  con el promedio entre  ( mes_anterior, mes_posterior )
# en honor a Velen Pennini,  honorable y fiel centinela de la Estadistica Clasica

Corregir_VelenPennini  <- function( dataset )
{
  CorregirCampoMes( "thomebanking", c(201801,202006) )
  CorregirCampoMes( "chomebanking_transacciones", c(201801, 201910, 202006) )
  CorregirCampoMes( "tcallcenter", c(201801, 201806, 202006) )
  CorregirCampoMes( "ccallcenter_transacciones", c(201801, 201806, 202006) )
  CorregirCampoMes( "cprestamos_personales", c(201801,202006) )
  CorregirCampoMes( "mprestamos_personales", c(201801,202006) )
  CorregirCampoMes( "mprestamos_hipotecarios", c(201801,202006) )
  CorregirCampoMes( "ccajas_transacciones", c(201801,202006) )
  CorregirCampoMes( "ccajas_consultas", c(201801,202006) )
  CorregirCampoMes( "ccajas_depositos", c(201801,202006) )
  CorregirCampoMes( "ccajas_extracciones", c(201801,202006) )
  CorregirCampoMes( "ccajas_otras", c(201801,202006) )

  CorregirCampoMes( "ctarjeta_visa_debitos_automaticos", c(201904) )
  CorregirCampoMes( "mttarjeta_visa_debitos_automaticos", c(201904,201905) )
  CorregirCampoMes( "Visa_mfinanciacion_limite", c(201904) )

  CorregirCampoMes( "mrentabilidad", c(201905, 201910, 202006) )
  CorregirCampoMes( "mrentabilidad_annual", c(201905, 201910, 202006) )
  CorregirCampoMes( "mcomisiones", c(201905, 201910, 202006) )
  CorregirCampoMes( "mpasivos_margen", c(201905, 201910, 202006) )
  CorregirCampoMes( "mactivos_margen", c(201905, 201910, 202006) )
  CorregirCampoMes( "ccomisiones_otras", c(201905, 201910, 202006) )
  CorregirCampoMes( "mcomisiones_otras", c(201905, 201910, 202006) )

  CorregirCampoMes( "ctarjeta_visa_descuentos", c(201910) )
  CorregirCampoMes( "ctarjeta_master_descuentos", c(201910) )
  CorregirCampoMes( "mtarjeta_visa_descuentos", c(201910) )
  CorregirCampoMes( "mtarjeta_master_descuentos", c(201910) )
  CorregirCampoMes( "ccajeros_propios_descuentos", c(201910) )
  CorregirCampoMes( "mcajeros_propios_descuentos", c(201910) )

  CorregirCampoMes( "cliente_vip", c(201911) )

  CorregirCampoMes( "active_quarter", c(202006) )
  CorregirCampoMes( "mcuentas_saldo", c(202006) )
  CorregirCampoMes( "ctarjeta_debito_transacciones", c(202006) )
  CorregirCampoMes( "mautoservicio", c(202006) )
  CorregirCampoMes( "ctarjeta_visa_transacciones", c(202006) )
  CorregirCampoMes( "ctarjeta_visa_transacciones", c(202006) )
  CorregirCampoMes( "cextraccion_autoservicio", c(202006) )
  CorregirCampoMes( "mextraccion_autoservicio", c(202006) )
  CorregirCampoMes( "ccheques_depositados", c(202006) )
  CorregirCampoMes( "mcheques_depositados", c(202006) )
  CorregirCampoMes( "mcheques_emitidos", c(202006) )
  CorregirCampoMes( "mcheques_emitidos", c(202006) )
  CorregirCampoMes( "ccheques_depositados_rechazados", c(202006) )
  CorregirCampoMes( "mcheques_depositados_rechazados", c(202006) )
  CorregirCampoMes( "ccheques_emitidos_rechazados", c(202006) )
  CorregirCampoMes( "mcheques_emitidos_rechazados", c(202006) )
  CorregirCampoMes( "catm_trx", c(202006) )
  CorregirCampoMes( "matm", c(202006) )
  CorregirCampoMes( "catm_trx_other", c(202006) )
  CorregirCampoMes( "matm_other", c(202006) )
  CorregirCampoMes( "tmobile_app", c(202006) )
  CorregirCampoMes( "cmobile_app_trx", c(202006) )

  #CorregirCampoMes( "internet",    c(201801, 202006, 202010, 202011, 202012, 202101, 202102, 202103) )
  CorregirCampoMes( "tmobile_app", c(202006, 202009, 202010, 202011, 202012, 202101, 202102, 202103) )
}
#------------------------------------------------------------------------------

Corregir_MachineLearning  <- function( dataset )
{
  gc()
  #acomodo los errores del dataset

  dataset[ foto_mes==201901,  ctransferencias_recibidas  := NA ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas  := NA ]

  dataset[ foto_mes==201902,  ctransferencias_recibidas  := NA ]
  dataset[ foto_mes==201902,  mtransferencias_recibidas  := NA ]

  dataset[ foto_mes==201903,  ctransferencias_recibidas  := NA ]
  dataset[ foto_mes==201903,  mtransferencias_recibidas  := NA ]

  dataset[ foto_mes==201904,  ctransferencias_recibidas  := NA ]
  dataset[ foto_mes==201904,  mtransferencias_recibidas  := NA ]
  dataset[ foto_mes==201904,  ctarjeta_visa_debitos_automaticos  :=  NA ]
  dataset[ foto_mes==201904,  mttarjeta_visa_debitos_automaticos := NA ]
  dataset[ foto_mes==201904,  Visa_mfinanciacion_limite := NA ]

  dataset[ foto_mes==201905,  ctransferencias_recibidas  := NA ]
  dataset[ foto_mes==201905,  mtransferencias_recibidas  := NA ]
  dataset[ foto_mes==201905,  mrentabilidad     := NA ]
  dataset[ foto_mes==201905,  mrentabilidad_annual     := NA ]
  dataset[ foto_mes==201905,  mcomisiones      := NA ]
  dataset[ foto_mes==201905,  mpasivos_margen  := NA ]
  dataset[ foto_mes==201905,  mactivos_margen  := NA ]
  dataset[ foto_mes==201905,  ctarjeta_visa_debitos_automaticos  := NA ]
  dataset[ foto_mes==201905,  ccomisiones_otras := NA ]
  dataset[ foto_mes==201905,  mcomisiones_otras := NA ]

  dataset[ foto_mes==201910,  mpasivos_margen   := NA ]
  dataset[ foto_mes==201910,  mactivos_margen   := NA ]
  dataset[ foto_mes==201910,  ccomisiones_otras := NA ]
  dataset[ foto_mes==201910,  mcomisiones_otras := NA ]
  dataset[ foto_mes==201910,  mcomisiones       := NA ]
  dataset[ foto_mes==201910,  mrentabilidad     := NA ]
  dataset[ foto_mes==201910,  mrentabilidad_annual        := NA ]
  dataset[ foto_mes==201910,  chomebanking_transacciones  := NA ]
  dataset[ foto_mes==201910,  ctarjeta_visa_descuentos    := NA ]
  dataset[ foto_mes==201910,  ctarjeta_master_descuentos  := NA ]
  dataset[ foto_mes==201910,  mtarjeta_visa_descuentos    := NA ]
  dataset[ foto_mes==201910,  mtarjeta_master_descuentos  := NA ]
  dataset[ foto_mes==201910,  ccajeros_propios_descuentos := NA ]
  dataset[ foto_mes==201910,  mcajeros_propios_descuentos := NA ]

  dataset[ foto_mes==202001,  cliente_vip   := NA ]

  dataset[ foto_mes==202006,  active_quarter   := NA ]
  #dataset[ foto_mes==202006,  internet   := NA ]
  dataset[ foto_mes==202006,  mrentabilidad   := NA ]
  dataset[ foto_mes==202006,  mrentabilidad_annual   := NA ]
  dataset[ foto_mes==202006,  mcomisiones   := NA ]
  dataset[ foto_mes==202006,  mactivos_margen   := NA ]
  dataset[ foto_mes==202006,  mpasivos_margen   := NA ]
  dataset[ foto_mes==202006,  mcuentas_saldo   := NA ]
  dataset[ foto_mes==202006,  ctarjeta_debito_transacciones  := NA ]
  dataset[ foto_mes==202006,  mautoservicio   := NA ]
  dataset[ foto_mes==202006,  ctarjeta_visa_transacciones   := NA ]
  dataset[ foto_mes==202006,  mtarjeta_visa_consumo   := NA ]
  dataset[ foto_mes==202006,  ctarjeta_master_transacciones  := NA ]
  dataset[ foto_mes==202006,  mtarjeta_master_consumo   := NA ]
  dataset[ foto_mes==202006,  ccomisiones_otras   := NA ]
  dataset[ foto_mes==202006,  mcomisiones_otras   := NA ]
  dataset[ foto_mes==202006,  cextraccion_autoservicio   := NA ]
  dataset[ foto_mes==202006,  mextraccion_autoservicio   := NA ]
  dataset[ foto_mes==202006,  ccheques_depositados   := NA ]
  dataset[ foto_mes==202006,  mcheques_depositados   := NA ]
  dataset[ foto_mes==202006,  ccheques_emitidos   := NA ]
  dataset[ foto_mes==202006,  mcheques_emitidos   := NA ]
  dataset[ foto_mes==202006,  ccheques_depositados_rechazados   := NA ]
  dataset[ foto_mes==202006,  mcheques_depositados_rechazados   := NA ]
  dataset[ foto_mes==202006,  ccheques_emitidos_rechazados   := NA ]
  dataset[ foto_mes==202006,  mcheques_emitidos_rechazados   := NA ]
  dataset[ foto_mes==202006,  tcallcenter   := NA ]
  dataset[ foto_mes==202006,  ccallcenter_transacciones   := NA ]
  dataset[ foto_mes==202006,  thomebanking   := NA ]
  dataset[ foto_mes==202006,  chomebanking_transacciones   := NA ]
  dataset[ foto_mes==202006,  ccajas_transacciones   := NA ]
  dataset[ foto_mes==202006,  ccajas_consultas   := NA ]
  dataset[ foto_mes==202006,  ccajas_depositos   := NA ]
  dataset[ foto_mes==202006,  ccajas_extracciones   := NA ]
  dataset[ foto_mes==202006,  ccajas_otras   := NA ]
  dataset[ foto_mes==202006,  catm_trx   := NA ]
  dataset[ foto_mes==202006,  matm   := NA ]
  dataset[ foto_mes==202006,  catm_trx_other   := NA ]
  dataset[ foto_mes==202006,  matm_other   := NA ]
  dataset[ foto_mes==202006,  ctrx_quarter   := NA ]
  dataset[ foto_mes==202006,  tmobile_app   := NA ]
  dataset[ foto_mes==202006,  cmobile_app_trx   := NA ]


  #dataset[ foto_mes==202010,  internet  := NA ]
  #dataset[ foto_mes==202011,  internet  := NA ]
  #dataset[ foto_mes==202012,  internet  := NA ]
  #dataset[ foto_mes==202101,  internet  := NA ]
  #dataset[ foto_mes==202102,  internet  := NA ]
  #dataset[ foto_mes==202103,  internet  := NA ]

  dataset[ foto_mes==202009,  tmobile_app  := NA ]
  dataset[ foto_mes==202010,  tmobile_app  := NA ]
  dataset[ foto_mes==202011,  tmobile_app  := NA ]
  dataset[ foto_mes==202012,  tmobile_app  := NA ]
  dataset[ foto_mes==202101,  tmobile_app  := NA ]
  dataset[ foto_mes==202102,  tmobile_app  := NA ]
  dataset[ foto_mes==202103,  tmobile_app  := NA ]

}
#------------------------------------------------------------------------------
#Esta es la parte que los alumnos deben desplegar todo su ingenio

AgregarVariables  <- function( dataset )
{
  gc()
  #INICIO de la seccion donde se deben hacer cambios con variables nuevas

  #creo un ctr_quarter que tenga en cuenta cuando los clientes hace 3 menos meses que estan
  dataset[  , ctrx_quarter_normalizado := ctrx_quarter ]
  dataset[ cliente_antiguedad==1 , ctrx_quarter_normalizado := ctrx_quarter * 5 ]
  dataset[ cliente_antiguedad==2 , ctrx_quarter_normalizado := ctrx_quarter * 2 ]
  dataset[ cliente_antiguedad==3 , ctrx_quarter_normalizado := ctrx_quarter * 1.2 ]

  #variable extraida de una tesis de maestria de Irlanda
  dataset[  , mpayroll_sobre_edad  := mpayroll / cliente_edad ]

  #se crean los nuevos campos para MasterCard  y Visa, teniendo en cuenta los NA's
  #varias formas de combinar Visa_status y Master_status
  dataset[ , mv_status01       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
  dataset[ , mv_status02       := Master_status +  Visa_status ]
  dataset[ , mv_status03       := pmax( ifelse( is.na(Master_status), 10, Master_status) , ifelse( is.na(Visa_status), 10, Visa_status) ) ]
  dataset[ , mv_status04       := ifelse( is.na(Master_status), 10, Master_status)  +  ifelse( is.na(Visa_status), 10, Visa_status)  ]
  dataset[ , mv_status05       := ifelse( is.na(Master_status), 10, Master_status)  +  100*ifelse( is.na(Visa_status), 10, Visa_status)  ]

  dataset[ , mv_status06       := ifelse( is.na(Visa_status), 
                                          ifelse( is.na(Master_status), 10, Master_status), 
                                          Visa_status)  ]

  dataset[ , mv_status07       := ifelse( is.na(Master_status), 
                                          ifelse( is.na(Visa_status), 10, Visa_status), 
                                          Master_status)  ]


  #combino MasterCard y Visa
  dataset[ , mv_mfinanciacion_limite := rowSums( cbind( Master_mfinanciacion_limite,  Visa_mfinanciacion_limite) , na.rm=TRUE ) ]

  dataset[ , mv_Fvencimiento         := pmin( Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE) ]
  dataset[ , mv_Finiciomora          := pmin( Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE) ]
  dataset[ , mv_msaldototal          := rowSums( cbind( Master_msaldototal,  Visa_msaldototal) , na.rm=TRUE ) ]
  dataset[ , mv_msaldopesos          := rowSums( cbind( Master_msaldopesos,  Visa_msaldopesos) , na.rm=TRUE ) ]
  dataset[ , mv_msaldodolares        := rowSums( cbind( Master_msaldodolares,  Visa_msaldodolares) , na.rm=TRUE ) ]
  dataset[ , mv_mconsumospesos       := rowSums( cbind( Master_mconsumospesos,  Visa_mconsumospesos) , na.rm=TRUE ) ]
  dataset[ , mv_mconsumosdolares     := rowSums( cbind( Master_mconsumosdolares,  Visa_mconsumosdolares) , na.rm=TRUE ) ]
  dataset[ , mv_mlimitecompra        := rowSums( cbind( Master_mlimitecompra,  Visa_mlimitecompra) , na.rm=TRUE ) ]
  dataset[ , mv_madelantopesos       := rowSums( cbind( Master_madelantopesos,  Visa_madelantopesos) , na.rm=TRUE ) ]
  dataset[ , mv_madelantodolares     := rowSums( cbind( Master_madelantodolares,  Visa_madelantodolares) , na.rm=TRUE ) ]
  dataset[ , mv_fultimo_cierre       := pmax( Master_fultimo_cierre, Visa_fultimo_cierre, na.rm = TRUE) ]
  dataset[ , mv_mpagado              := rowSums( cbind( Master_mpagado,  Visa_mpagado) , na.rm=TRUE ) ]
  dataset[ , mv_mpagospesos          := rowSums( cbind( Master_mpagospesos,  Visa_mpagospesos) , na.rm=TRUE ) ]
  dataset[ , mv_mpagosdolares        := rowSums( cbind( Master_mpagosdolares,  Visa_mpagosdolares) , na.rm=TRUE ) ]
  dataset[ , mv_fechaalta            := pmax( Master_fechaalta, Visa_fechaalta, na.rm = TRUE) ]
  dataset[ , mv_mconsumototal        := rowSums( cbind( Master_mconsumototal,  Visa_mconsumototal) , na.rm=TRUE ) ]
  dataset[ , mv_cconsumos            := rowSums( cbind( Master_cconsumos,  Visa_cconsumos) , na.rm=TRUE ) ]
  dataset[ , mv_cadelantosefectivo   := rowSums( cbind( Master_cadelantosefectivo,  Visa_cadelantosefectivo) , na.rm=TRUE ) ]
  dataset[ , mv_mpagominimo          := rowSums( cbind( Master_mpagominimo,  Visa_mpagominimo) , na.rm=TRUE ) ]

  #a partir de aqui juego con la suma de Mastercard y Visa
  dataset[ , mvr_Master_mlimitecompra:= Master_mlimitecompra / mv_mlimitecompra ]
  dataset[ , mvr_Visa_mlimitecompra  := Visa_mlimitecompra / mv_mlimitecompra ]
  dataset[ , mvr_msaldototal         := mv_msaldototal / mv_mlimitecompra ]
  dataset[ , mvr_msaldopesos         := mv_msaldopesos / mv_mlimitecompra ]
  dataset[ , mvr_msaldopesos2        := mv_msaldopesos / mv_msaldototal ]
  dataset[ , mvr_msaldodolares       := mv_msaldodolares / mv_mlimitecompra ]
  dataset[ , mvr_msaldodolares2      := mv_msaldodolares / mv_msaldototal ]
  dataset[ , mvr_mconsumospesos      := mv_mconsumospesos / mv_mlimitecompra ]
  dataset[ , mvr_mconsumosdolares    := mv_mconsumosdolares / mv_mlimitecompra ]
  dataset[ , mvr_madelantopesos      := mv_madelantopesos / mv_mlimitecompra ]
  dataset[ , mvr_madelantodolares    := mv_madelantodolares / mv_mlimitecompra ]
  dataset[ , mvr_mpagado             := mv_mpagado / mv_mlimitecompra ]
  dataset[ , mvr_mpagospesos         := mv_mpagospesos / mv_mlimitecompra ]
  dataset[ , mvr_mpagosdolares       := mv_mpagosdolares / mv_mlimitecompra ]
  dataset[ , mvr_mconsumototal       := mv_mconsumototal  / mv_mlimitecompra ]
  dataset[ , mvr_mpagominimo         := mv_mpagominimo  / mv_mlimitecompra ]

  #Aqui debe usted agregar sus propias nuevas variables

  #valvula de seguridad para evitar valores infinitos
  #paso los infinitos a NULOS
  infinitos      <- lapply(names(dataset),function(.name) dataset[ , sum(is.infinite(get(.name)))])
  infinitos_qty  <- sum( unlist( infinitos) )
  if( infinitos_qty > 0 )
  {
    cat( "ATENCION, hay", infinitos_qty, "valores infinitos en tu dataset. Seran pasados a NA\n" )
    dataset[mapply(is.infinite, dataset)] <<- NA
  }


  #valvula de seguridad para evitar valores NaN  que es 0/0
  #paso los NaN a 0 , decision polemica si las hay
  #se invita a asignar un valor razonable segun la semantica del campo creado
  nans      <- lapply(names(dataset),function(.name) dataset[ , sum(is.nan(get(.name)))])
  nans_qty  <- sum( unlist( nans) )
  if( nans_qty > 0 )
  {
    cat( "ATENCION, hay", nans_qty, "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n" )
    cat( "Si no te gusta la decision, modifica a gusto el programa!\n\n")
    dataset[mapply(is.nan, dataset)] <<- 0
  }

}

#------------------------------------------------------------------------------
#se calculan para los 6 meses previos el minimo, maximo y tendencia calculada con cuadrados minimos
#la formula de calculo de la tendencia puede verse en https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
#para la maxíma velocidad esta funcion esta escrita en lenguaje C, y no en la porqueria de R o Python

cppFunction('NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde ) 
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) ) 
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      { 
        xsum  += x[h] ;
        ysum  += y[h] ; 
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ; 
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}')

#------------------------------------------------------------------------------
#calcula la tendencia de las variables cols de los ultimos 6 meses
#la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
#La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas  <- function( dataset, cols, ventana=6, tendencia=TRUE, minimo=TRUE, maximo=TRUE, promedio=TRUE, 
                                 ratioavg=FALSE, ratiomax=FALSE)
{
  gc()
  #Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion  <- ventana

  last  <- nrow( dataset )

  #creo el vector_desde que indica cada ventana
  #de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids   <- dataset$numero_de_cliente

  vector_desde  <- seq( -ventana_regresion+2,  nrow(dataset)-ventana_regresion+1 )
  vector_desde[ 1:ventana_regresion ]  <-  1

  for( i in 2:last )  if( vector_ids[ i-1 ] !=  vector_ids[ i ] ) {  vector_desde[i] <-  i }
  for( i in 2:last )  if( vector_desde[i] < vector_desde[i-1] )  {  vector_desde[i] <-  vector_desde[i-1] }

  for(  campo  in   cols )
  {
    nueva_col     <- fhistC( dataset[ , get(campo) ], vector_desde ) 

    if(tendencia)  dataset[ , paste0( campo, "_tend", ventana) := nueva_col[ (0*last +1):(1*last) ]  ]
    if(minimo)     dataset[ , paste0( campo, "_min", ventana)  := nueva_col[ (1*last +1):(2*last) ]  ]
    if(maximo)     dataset[ , paste0( campo, "_max", ventana)  := nueva_col[ (2*last +1):(3*last) ]  ]
    if(promedio)   dataset[ , paste0( campo, "_avg", ventana)  := nueva_col[ (3*last +1):(4*last) ]  ]
    if(ratioavg)   dataset[ , paste0( campo, "_ratioavg", ventana)  := get(campo) /nueva_col[ (3*last +1):(4*last) ]  ]
    if(ratiomax)   dataset[ , paste0( campo, "_ratiomax", ventana)  := get(campo) /nueva_col[ (2*last +1):(3*last) ]  ]
  }

}
#------------------------------------------------------------------------------
#agrega al dataset nuevas variables {0,1} que provienen de las hojas de un Random Forest

AgregaVarRandomForest  <- function( num.trees, max.depth, min.node.size, mtry)
{
  gc()
  dataset[ , clase01:= ifelse( clase_ternaria=="CONTINUA", 0, 1 ) ]

  campos_buenos  <- setdiff( colnames(dataset), c("clase_ternaria" ) )

  dataset_rf  <- copy( dataset[ , campos_buenos, with=FALSE] )
  azar  <- runif( nrow(dataset_rf) )
  dataset_rf[ , entrenamiento := as.integer( foto_mes>= 202009 &  foto_mes<= 202101 & ( clase01==1 | azar < 0.10 )) ]

  #imputo los nulos, ya que ranger no acepta nulos
  #Leo Breiman, ¿por que le temias a los nulos?
  dataset_rf  <- na.roughfix( dataset_rf )

  campos_buenos  <- setdiff( colnames(dataset_rf), c("clase_ternaria","entrenamiento" ) )
  modelo  <- ranger( formula= "clase01 ~ .",
                     data=  dataset_rf[ entrenamiento==1L, campos_buenos, with=FALSE  ] ,
                     classification= TRUE,
                     probability=   FALSE,
                     num.trees=     num.trees,
                     max.depth=     max.depth,
                     min.node.size= min.node.size,
                     mtry=          mtry
                   )

  rfhojas  <- predict( object= modelo, 
                       data= dataset_rf[ , campos_buenos, with=FALSE ],
                       predict.all= TRUE,    #entrega la prediccion de cada arbol
                       type= "terminalNodes" #entrega el numero de NODO el arbol
                     )

  for( arbol in 1:num.trees )
  {
    hojas_arbol  <- unique(  rfhojas$predictions[  , arbol  ] )

    for( pos in 1:length(hojas_arbol) )
    {
      nodo_id  <- hojas_arbol[ pos ]  #el numero de nodo de la hoja, estan salteados
      dataset[  ,  paste0( "rf_", sprintf( "%03d", arbol ), "_", sprintf( "%03d", nodo_id ) ) := 0L ]

      dataset[ which( rfhojas$predictions[ , arbol] == nodo_id ,  ), 
               paste0( "rf_", sprintf( "%03d", arbol ), "_", sprintf( "%03d", nodo_id ) ) := 1L ]
    }
  }

  rm( dataset_rf )
  dataset[ , clase01 := NULL ]

  gc()
}
#------------------------------------------------------------------------------
VPOS_CORTE  <- c()

fganancia_lgbm_meseta  <- function(probs, datos) 
{
  vlabels  <- get_field(datos, "label")
  vpesos   <- get_field(datos, "weight")

  tbl  <- as.data.table( list( "prob"=probs, "gan"= ifelse( vlabels==1 & vpesos > 1, 60000, -2000 ) ) )

  setorder( tbl, -prob )
  tbl[ , posicion := .I ]
  tbl[ , gan_acum :=  cumsum( gan ) ]
  setorder( tbl, -gan_acum )   #voy por la meseta

  gan  <- mean( tbl[ 1:500,  gan_acum] )  #meseta de tamaño 500

  pos_meseta  <- tbl[ 1:500,  median(posicion)]
  VPOS_CORTE  <<- c( VPOS_CORTE, pos_meseta )

  return( list( "name"= "ganancia", 
                "value"=  gan,
                "higher_better"= TRUE ) )
}
#------------------------------------------------------------------------------
#Elimina del dataset las variables que estan por debajo de la capa geologica de canaritos
#se llama varias veces, luego de agregar muchas variables nuevas, para ir reduciendo la cantidad de variables
# y así hacer lugar a nuevas variables importantes

GVEZ <- 1 

CanaritosAsesinos  <- function( canaritos_ratio=0.2 )
{
  gc()
  dataset[ , clase01:= ifelse( clase_ternaria=="CONTINUA", 0, 1 ) ]

  for( i  in 1:(ncol(dataset)*canaritos_ratio))  dataset[ , paste0("canarito", i ) :=  runif( nrow(dataset))]

  campos_buenos  <- setdiff( colnames(dataset), c("clase_ternaria","clase01", "foto_mes" ) )

  azar  <- runif( nrow(dataset) )
  dataset[ , entrenamiento := foto_mes>= 202008 &  foto_mes<= 202111  & ( clase01==1 | azar < 0.10 ) ]

  dtrain  <- lgb.Dataset( data=    data.matrix(  dataset[ entrenamiento==TRUE, campos_buenos, with=FALSE]),
                          label=   dataset[ entrenamiento==TRUE, clase01],
                          weight=  dataset[ entrenamiento==TRUE, ifelse(clase_ternaria=="BAJA+2", 1.0000001, 1.0)],
                          free_raw_data= FALSE
                        )

  dvalid  <- lgb.Dataset( data=    data.matrix(  dataset[ foto_mes==202101, campos_buenos, with=FALSE]),
                          label=   dataset[ foto_mes==202101, clase01],
                          weight=  dataset[ foto_mes==202101, ifelse(clase_ternaria=="BAJA+2", 1.0000001, 1.0)],
                          free_raw_data= FALSE
                          )


  param <- list( objective= "binary",
                 metric= "custom",
                 first_metric_only= TRUE,
                 boost_from_average= TRUE,
                 feature_pre_filter= FALSE,
                 verbosity= -100,
                 seed= 999983,
                 max_depth=  -1,         # -1 significa no limitar,  por ahora lo dejo fijo
                 min_gain_to_split= 0.0, #por ahora, lo dejo fijo
                 lambda_l1= 0.0,         #por ahora, lo dejo fijo
                 lambda_l2= 0.0,         #por ahora, lo dejo fijo
                 max_bin= 31,            #por ahora, lo dejo fijo
                 num_iterations= 9999,   #un numero muy grande, lo limita early_stopping_rounds
                 force_row_wise= TRUE,    #para que los alumnos no se atemoricen con tantos warning
                 learning_rate= 0.065, 
                 feature_fraction= 1.0,   #lo seteo en 1 para que las primeras variables del dataset no se vean opacadas
                 min_data_in_leaf= 260,
                 num_leaves= 60,
                 early_stopping_rounds= 200 )

  modelo  <- lgb.train( data= dtrain,
                        valids= list( valid= dvalid ),
                        eval= fganancia_lgbm_meseta,
                        param= param,
                        verbose= -100 )

  tb_importancia  <- lgb.importance( model= modelo )
  tb_importancia[  , pos := .I ]

  fwrite( tb_importancia, 
          file= paste0( "impo_", GVEZ ,".txt"),
          sep= "\t" )

  GVEZ  <<- GVEZ + 1

  umbral  <- tb_importancia[ Feature %like% "canarito", median(pos) + 2*sd(pos) ]  #Atencion corto en la mediana mas DOS desvios!!

  col_utiles  <- tb_importancia[ pos < umbral & !( Feature %like% "canarito"),  Feature ]
  col_utiles  <-  unique( c( col_utiles,  c("numero_de_cliente","foto_mes","clase_ternaria","mes") ) )
  col_inutiles  <- setdiff( colnames(dataset), col_utiles )

  dataset[  ,  (col_inutiles) := NULL ]

}
#------------------------------------------------------------------------------
#agrega para cada columna de cols una nueva variable _rank  que es un numero entre 0 y 1  del ranking de esa variable ese mes

Rankeador  <- function( cols )
{
  gc()
  sufijo  <- "_rank" 

  for( vcol in cols )
  {
     dataset[ , paste0( vcol, sufijo) := frank( get(vcol), ties.method= "random")/ .N, 
                by= foto_mes ]
  }
}
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------
#Aqui empieza el programa

setwd( "~/buckets/b1/" )

#cargo el dataset
dataset  <- fread( "./datasets/competencia1_historia_2022.csv.gz" )

#Elimino los campos problematicos
dataset[  , internet := NULL ]

#creo la carpeta donde va el experimento
# FE  representa  Feature Engineering
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/FE8150/", showWarnings = FALSE )
setwd("./exp/FE8150/")   #Establezco el Working Directory DEL EXPERIMENTO



#corrijo los  < foto_mes, campo >  que fueron pisados con cero
# Atencion, cambiar si se desea el metodo VelenPennini
Corregir_MachineLearning( dataset )  
# Corregir_VelenPennini( dataset )


#Agrego las variables manuales al dataset
AgregarVariables( dataset )


#--------------------------------------
#estas son las columnas a las que se puede agregar lags o media moviles ( todas menos las obvias )
cols_lagueables  <- copy(  setdiff( colnames(dataset), c("numero_de_cliente", "foto_mes", "clase_ternaria")  ) )

#ordeno el dataset por <numero_de_cliente, foto_mes> para poder hacer lags
#  es MUY  importante esta linea
setorder( dataset, numero_de_cliente, foto_mes )

#creo los campos lags de orden 1
dataset[ , paste0( cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"), 
           by= numero_de_cliente, 
           .SDcols= cols_lagueables ]

#agrego los delta lags
for( vcol in cols_lagueables )
{
  dataset[ , paste0(vcol, "_delta1") := get(vcol)  - get(paste0( vcol, "_lag2"))  ]
}

#creo los campos lags de orden 2
dataset[ , paste0( cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag2"), 
         by= numero_de_cliente, 
         .SDcols= cols_lagueables ]

#agrego los delta lags
for( vcol in cols_lagueables )
{
  dataset[ , paste0(vcol, "_delta2") := get(vcol)  - get(paste0( vcol, "_lag2"))  ]
}
#creo los campos lags de orden 3
dataset[ , paste0( cols_lagueables, "_lag3") := shift(.SD, 3, NA, "lag3"), 
         by= numero_de_cliente, 
         .SDcols= cols_lagueables ]

#agrego los delta lags
for( vcol in cols_lagueables )
{
  dataset[ , paste0(vcol, "_delta3") := get(vcol)  - get(paste0( vcol, "_lag3"))  ]


#--------------------------------------
#agrego las tendencias

TendenciaYmuchomas( dataset, 
                    cols= cols_lagueables,
                    ventana=   6,      # 6 meses de historia
                    tendencia= TRUE,
                    minimo=    FALSE,
                    maximo=    FALSE,
                    promedio=  TRUE,
                    ratioavg=  FALSE,
                    ratiomax=  FALSE  )


#------------------------------------------------------------------------------
#Agrego variables a partir de las hojas de un Random Forest

AgregaVarRandomForest( num.trees = 10,
                       max.depth = 5,
                       min.node.size = 500,
                       mtry = 15 )

gc()

#------------------------------------------------------------------------------
#Elimino las variables que no son tan importantes en el dataset

ncol( dataset )
CanaritosAsesinos( canaritos_ratio = 0.3 )
ncol( dataset )

#------------------------------------------------------------------------------
#grabo el dataset
fwrite( dataset,
        "dataset_7130.csv.gz",
        logical01= TRUE,
        sep= "," )


Pero fallo ampliamente ejecutandose.. 

Estamos intentar correr la version del còdigo TAL CUAL VIENE a ver si logramos sacar algo. REquerí ayuda de terceros para poder crear la maquina del tamaño correcto. En el intermedio me acusaron de minar cryptos con las VMs

#grabo el dataset
fwrite( dataset,
        "dataset_7130lob.csv.gz",
        logical01= TRUE,
        sep= "," )

Al fin estaba creado. Aprovecharemos el tiempo para prender otra maquina para 633 e ir generando algunos hiperparametros para esta nueva creaciòn conformista e incompleta de lo que se necesita para realmente lograr algo.

A su vez, podriamos intentar correr de nuevo otro. 

In [ ]:
# source( "~/labo/src/FeatureEngineering/z815_FE_final.r")
#Necesita para correr en Google Cloud
# 256 GB de memoria RAM
# 256 GB de espacio en el disco local
#   8 vCPU


#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("Rcpp")

require("ranger")
require("randomForest")  #solo se usa para imputar nulos

require("lightgbm")

options(error = function() { 
  traceback(20); 
  options(error = NULL); 
  stop("exiting after script error") 
})


#------------------------------------------------------------------------------

CorregirCampoMes  <- function( pcampo, pmeses )
{
  tbl <- dataset[  ,  list( "v1" = shift( get(pcampo), 1, type="lag" ),
                            "v2" = shift( get(pcampo), 1, type="lead" )
  ), 
  by=numero_de_cliente ]
  
  tbl[ , numero_de_cliente := NULL ]
  tbl[ , promedio := rowMeans( tbl,  na.rm=TRUE ) ]
  
  dataset[ ,
           paste0(pcampo) := ifelse( !(foto_mes %in% pmeses),
                                     get( pcampo),
                                     tbl$promedio ) ]
}
#------------------------------------------------------------------------------
# reemplaza cada variable ROTA  (variable, foto_mes)  con el promedio entre  ( mes_anterior, mes_posterior )
# en honor a Velen Pennini,  honorable y fiel centinela de la Estadistica Clasica

Corregir_VelenPennini  <- function( dataset )
{
  CorregirCampoMes( "thomebanking", c(201801,202006) )
  CorregirCampoMes( "chomebanking_transacciones", c(201801, 201910, 202006) )
  CorregirCampoMes( "tcallcenter", c(201801, 201806, 202006) )
  CorregirCampoMes( "ccallcenter_transacciones", c(201801, 201806, 202006) )
  CorregirCampoMes( "cprestamos_personales", c(201801,202006) )
  CorregirCampoMes( "mprestamos_personales", c(201801,202006) )
  CorregirCampoMes( "mprestamos_hipotecarios", c(201801,202006) )
  CorregirCampoMes( "ccajas_transacciones", c(201801,202006) )
  CorregirCampoMes( "ccajas_consultas", c(201801,202006) )
  CorregirCampoMes( "ccajas_depositos", c(201801,202006) )
  CorregirCampoMes( "ccajas_extracciones", c(201801,202006) )
  CorregirCampoMes( "ccajas_otras", c(201801,202006) )
  
  CorregirCampoMes( "ctarjeta_visa_debitos_automaticos", c(201904) )
  CorregirCampoMes( "mttarjeta_visa_debitos_automaticos", c(201904,201905) )
  CorregirCampoMes( "Visa_mfinanciacion_limite", c(201904) )
  
  CorregirCampoMes( "mrentabilidad", c(201905, 201910, 202006) )
  CorregirCampoMes( "mrentabilidad_annual", c(201905, 201910, 202006) )
  CorregirCampoMes( "mcomisiones", c(201905, 201910, 202006) )
  CorregirCampoMes( "mpasivos_margen", c(201905, 201910, 202006) )
  CorregirCampoMes( "mactivos_margen", c(201905, 201910, 202006) )
  CorregirCampoMes( "ccomisiones_otras", c(201905, 201910, 202006) )
  CorregirCampoMes( "mcomisiones_otras", c(201905, 201910, 202006) )
  
  CorregirCampoMes( "ctarjeta_visa_descuentos", c(201910) )
  CorregirCampoMes( "ctarjeta_master_descuentos", c(201910) )
  CorregirCampoMes( "mtarjeta_visa_descuentos", c(201910) )
  CorregirCampoMes( "mtarjeta_master_descuentos", c(201910) )
  CorregirCampoMes( "ccajeros_propios_descuentos", c(201910) )
  CorregirCampoMes( "mcajeros_propios_descuentos", c(201910) )
  
  CorregirCampoMes( "cliente_vip", c(201911) )
  
  CorregirCampoMes( "active_quarter", c(202006) )
  CorregirCampoMes( "mcuentas_saldo", c(202006) )
  CorregirCampoMes( "ctarjeta_debito_transacciones", c(202006) )
  CorregirCampoMes( "mautoservicio", c(202006) )
  CorregirCampoMes( "ctarjeta_visa_transacciones", c(202006) )
  CorregirCampoMes( "ctarjeta_visa_transacciones", c(202006) )
  CorregirCampoMes( "cextraccion_autoservicio", c(202006) )
  CorregirCampoMes( "mextraccion_autoservicio", c(202006) )
  CorregirCampoMes( "ccheques_depositados", c(202006) )
  CorregirCampoMes( "mcheques_depositados", c(202006) )
  CorregirCampoMes( "mcheques_emitidos", c(202006) )
  CorregirCampoMes( "mcheques_emitidos", c(202006) )
  CorregirCampoMes( "ccheques_depositados_rechazados", c(202006) )
  CorregirCampoMes( "mcheques_depositados_rechazados", c(202006) )
  CorregirCampoMes( "ccheques_emitidos_rechazados", c(202006) )
  CorregirCampoMes( "mcheques_emitidos_rechazados", c(202006) )
  CorregirCampoMes( "catm_trx", c(202006) )
  CorregirCampoMes( "matm", c(202006) )
  CorregirCampoMes( "catm_trx_other", c(202006) )
  CorregirCampoMes( "matm_other", c(202006) )
  CorregirCampoMes( "tmobile_app", c(202006) )
  CorregirCampoMes( "cmobile_app_trx", c(202006) )
  
  #CorregirCampoMes( "internet",    c(201801, 202006, 202010, 202011, 202012, 202101, 202102, 202103) )
  CorregirCampoMes( "tmobile_app", c(202006, 202009, 202010, 202011, 202012, 202101, 202102, 202103) )
}
#------------------------------------------------------------------------------

Corregir_MachineLearning  <- function( dataset )
{
  gc()
  #acomodo los errores del dataset
  
  dataset[ foto_mes==201901,  ctransferencias_recibidas  := NA ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas  := NA ]
  
  dataset[ foto_mes==201902,  ctransferencias_recibidas  := NA ]
  dataset[ foto_mes==201902,  mtransferencias_recibidas  := NA ]
  
  dataset[ foto_mes==201903,  ctransferencias_recibidas  := NA ]
  dataset[ foto_mes==201903,  mtransferencias_recibidas  := NA ]
  
  dataset[ foto_mes==201904,  ctransferencias_recibidas  := NA ]
  dataset[ foto_mes==201904,  mtransferencias_recibidas  := NA ]
  dataset[ foto_mes==201904,  ctarjeta_visa_debitos_automaticos  :=  NA ]
  dataset[ foto_mes==201904,  mttarjeta_visa_debitos_automaticos := NA ]
  dataset[ foto_mes==201904,  Visa_mfinanciacion_limite := NA ]
  
  dataset[ foto_mes==201905,  ctransferencias_recibidas  := NA ]
  dataset[ foto_mes==201905,  mtransferencias_recibidas  := NA ]
  dataset[ foto_mes==201905,  mrentabilidad     := NA ]
  dataset[ foto_mes==201905,  mrentabilidad_annual     := NA ]
  dataset[ foto_mes==201905,  mcomisiones      := NA ]
  dataset[ foto_mes==201905,  mpasivos_margen  := NA ]
  dataset[ foto_mes==201905,  mactivos_margen  := NA ]
  dataset[ foto_mes==201905,  ctarjeta_visa_debitos_automaticos  := NA ]
  dataset[ foto_mes==201905,  ccomisiones_otras := NA ]
  dataset[ foto_mes==201905,  mcomisiones_otras := NA ]
  
  dataset[ foto_mes==201910,  mpasivos_margen   := NA ]
  dataset[ foto_mes==201910,  mactivos_margen   := NA ]
  dataset[ foto_mes==201910,  ccomisiones_otras := NA ]
  dataset[ foto_mes==201910,  mcomisiones_otras := NA ]
  dataset[ foto_mes==201910,  mcomisiones       := NA ]
  dataset[ foto_mes==201910,  mrentabilidad     := NA ]
  dataset[ foto_mes==201910,  mrentabilidad_annual        := NA ]
  dataset[ foto_mes==201910,  chomebanking_transacciones  := NA ]
  dataset[ foto_mes==201910,  ctarjeta_visa_descuentos    := NA ]
  dataset[ foto_mes==201910,  ctarjeta_master_descuentos  := NA ]
  dataset[ foto_mes==201910,  mtarjeta_visa_descuentos    := NA ]
  dataset[ foto_mes==201910,  mtarjeta_master_descuentos  := NA ]
  dataset[ foto_mes==201910,  ccajeros_propios_descuentos := NA ]
  dataset[ foto_mes==201910,  mcajeros_propios_descuentos := NA ]
  
  dataset[ foto_mes==202001,  cliente_vip   := NA ]
  
  dataset[ foto_mes==202006,  active_quarter   := NA ]
  #dataset[ foto_mes==202006,  internet   := NA ]
  dataset[ foto_mes==202006,  mrentabilidad   := NA ]
  dataset[ foto_mes==202006,  mrentabilidad_annual   := NA ]
  dataset[ foto_mes==202006,  mcomisiones   := NA ]
  dataset[ foto_mes==202006,  mactivos_margen   := NA ]
  dataset[ foto_mes==202006,  mpasivos_margen   := NA ]
  dataset[ foto_mes==202006,  mcuentas_saldo   := NA ]
  dataset[ foto_mes==202006,  ctarjeta_debito_transacciones  := NA ]
  dataset[ foto_mes==202006,  mautoservicio   := NA ]
  dataset[ foto_mes==202006,  ctarjeta_visa_transacciones   := NA ]
  dataset[ foto_mes==202006,  mtarjeta_visa_consumo   := NA ]
  dataset[ foto_mes==202006,  ctarjeta_master_transacciones  := NA ]
  dataset[ foto_mes==202006,  mtarjeta_master_consumo   := NA ]
  dataset[ foto_mes==202006,  ccomisiones_otras   := NA ]
  dataset[ foto_mes==202006,  mcomisiones_otras   := NA ]
  dataset[ foto_mes==202006,  cextraccion_autoservicio   := NA ]
  dataset[ foto_mes==202006,  mextraccion_autoservicio   := NA ]
  dataset[ foto_mes==202006,  ccheques_depositados   := NA ]
  dataset[ foto_mes==202006,  mcheques_depositados   := NA ]
  dataset[ foto_mes==202006,  ccheques_emitidos   := NA ]
  dataset[ foto_mes==202006,  mcheques_emitidos   := NA ]
  dataset[ foto_mes==202006,  ccheques_depositados_rechazados   := NA ]
  dataset[ foto_mes==202006,  mcheques_depositados_rechazados   := NA ]
  dataset[ foto_mes==202006,  ccheques_emitidos_rechazados   := NA ]
  dataset[ foto_mes==202006,  mcheques_emitidos_rechazados   := NA ]
  dataset[ foto_mes==202006,  tcallcenter   := NA ]
  dataset[ foto_mes==202006,  ccallcenter_transacciones   := NA ]
  dataset[ foto_mes==202006,  thomebanking   := NA ]
  dataset[ foto_mes==202006,  chomebanking_transacciones   := NA ]
  dataset[ foto_mes==202006,  ccajas_transacciones   := NA ]
  dataset[ foto_mes==202006,  ccajas_consultas   := NA ]
  dataset[ foto_mes==202006,  ccajas_depositos   := NA ]
  dataset[ foto_mes==202006,  ccajas_extracciones   := NA ]
  dataset[ foto_mes==202006,  ccajas_otras   := NA ]
  dataset[ foto_mes==202006,  catm_trx   := NA ]
  dataset[ foto_mes==202006,  matm   := NA ]
  dataset[ foto_mes==202006,  catm_trx_other   := NA ]
  dataset[ foto_mes==202006,  matm_other   := NA ]
  dataset[ foto_mes==202006,  ctrx_quarter   := NA ]
  dataset[ foto_mes==202006,  tmobile_app   := NA ]
  dataset[ foto_mes==202006,  cmobile_app_trx   := NA ]
  
  
  #dataset[ foto_mes==202010,  internet  := NA ]
  #dataset[ foto_mes==202011,  internet  := NA ]
  #dataset[ foto_mes==202012,  internet  := NA ]
  #dataset[ foto_mes==202101,  internet  := NA ]
  #dataset[ foto_mes==202102,  internet  := NA ]
  #dataset[ foto_mes==202103,  internet  := NA ]
  
  dataset[ foto_mes==202009,  tmobile_app  := NA ]
  dataset[ foto_mes==202010,  tmobile_app  := NA ]
  dataset[ foto_mes==202011,  tmobile_app  := NA ]
  dataset[ foto_mes==202012,  tmobile_app  := NA ]
  dataset[ foto_mes==202101,  tmobile_app  := NA ]
  dataset[ foto_mes==202102,  tmobile_app  := NA ]
  dataset[ foto_mes==202103,  tmobile_app  := NA ]
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 189.6101 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 189.6101 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 196.7501 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 196.7501 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 205.9571 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 205.9571 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 213.0517 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 213.0517 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 219.5691 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 219.5691 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 225.537 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 225.537 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 225.537 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 225.537 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 225.537 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 230.494 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 230.494 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 230.494 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 230.494 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 230.494 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 239.6077 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 239.6077 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 253.7102 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 253.7102 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 262.0661 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 262.0661 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 273.2158 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 273.2158 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 283.4442 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 283.4442 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 289.8299 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 289.8299 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 295.666 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 295.666 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 295.666 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 295.666 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 295.666 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 305.5515 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 305.5515 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 310.1243 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 310.1243 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 314.9087 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 314.9087 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 321.9738 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 321.9738 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 328.2014 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 328.2014 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 337.0632 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 337.0632 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 346.6207 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 346.6207 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 359.657 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 359.657 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 359.657 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 359.657 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 359.657 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 371.0211 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 371.0211 * 402 ]
  
  dataset[ foto_mes==201901,  mrentabilidad := mrentabilidad / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mrentabilidad_annual := mrentabilidad_annual / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones := mcomisiones / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mactivos_margen := mactivos_margen / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mpasivos_margen := mpasivos_margen / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente_adicional := mcuenta_corriente_adicional / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_corriente := mcuenta_corriente / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro := mcaja_ahorro / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_adicional := mcaja_ahorro_adicional / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcaja_ahorro_dolares := mcaja_ahorro_dolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcuentas_saldo := mcuentas_saldo / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mautoservicio := mautoservicio / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_consumo := mtarjeta_visa_consumo / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_consumo := mtarjeta_master_consumo / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_personales := mprestamos_personales / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_prendarios := mprestamos_prendarios / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mprestamos_hipotecarios := mprestamos_hipotecarios / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_dolares := mplazo_fijo_dolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mplazo_fijo_pesos := mplazo_fijo_pesos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  minversion1_pesos := minversion1_pesos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  minversion1_dolares := minversion1_dolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  minversion2 := minversion2 / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mpayroll := mpayroll / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mpayroll2 := mpayroll2 / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcuenta_debitos_automaticos := mcuenta_debitos_automaticos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mttarjeta_master_debitos_automaticos := mttarjeta_master_debitos_automaticos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mpagodeservicios := mpagodeservicios / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mpagomiscuentas := mpagomiscuentas / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcajeros_propios_descuentos := mcajeros_propios_descuentos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_visa_descuentos := mtarjeta_visa_descuentos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mtarjeta_master_descuentos := mtarjeta_master_descuentos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_mantenimiento := mcomisiones_mantenimiento / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcomisiones_otras := mcomisiones_otras / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mforex_buy := mforex_buy / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mforex_sell := mforex_sell / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_recibidas := mtransferencias_recibidas / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mtransferencias_emitidas := mtransferencias_emitidas / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mextraccion_autoservicio := mextraccion_autoservicio / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados := mcheques_depositados / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos := mcheques_emitidos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcheques_depositados_rechazados := mcheques_depositados_rechazados / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  mcheques_emitidos_rechazados := mcheques_emitidos_rechazados / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  matm := matm / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  matm_other := matm_other / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_mfinanciacion_limite := Master_mfinanciacion_limite / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldototal := Master_msaldototal / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldopesos := Master_msaldopesos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_msaldodolares := Master_msaldodolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumospesos := Master_mconsumospesos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumosdolares := Master_mconsumosdolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_mlimitecompra := Master_mlimitecompra / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantopesos := Master_madelantopesos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_madelantodolares := Master_madelantodolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagado := Master_mpagado / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagospesos := Master_mpagospesos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagosdolares := Master_mpagosdolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_mconsumototal := Master_mconsumototal / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Master_mpagominimo := Master_mpagominimo / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_mfinanciacion_limite := Visa_mfinanciacion_limite / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldototal := Visa_msaldototal / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldopesos := Visa_msaldopesos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_msaldodolares := Visa_msaldodolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumospesos := Visa_mconsumospesos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumosdolares := Visa_mconsumosdolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_mlimitecompra := Visa_mlimitecompra / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantopesos := Visa_madelantopesos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_madelantodolares := Visa_madelantodolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagado := Visa_mpagado / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagospesos := Visa_mpagospesos / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagosdolares := Visa_mpagosdolares / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_mconsumototal := Visa_mconsumototal / 385.8826 * 402 ]
  dataset[ foto_mes==201901,  Visa_mpagominimo := Visa_mpagominimo / 385.8826 * 402 ]
  
  
  
  
}
#------------------------------------------------------------------------------
#Esta es la parte que los alumnos deben desplegar todo su ingenio

AgregarVariables  <- function( dataset )
{
  gc()
  #INICIO de la seccion donde se deben hacer cambios con variables nuevas
  
  #creo un ctr_quarter que tenga en cuenta cuando los clientes hace 3 menos meses que estan
  dataset[  , ctrx_quarter_normalizado := ctrx_quarter ]
  dataset[ cliente_antiguedad==1 , ctrx_quarter_normalizado := ctrx_quarter * 5 ]
  dataset[ cliente_antiguedad==2 , ctrx_quarter_normalizado := ctrx_quarter * 2 ]
  dataset[ cliente_antiguedad==3 , ctrx_quarter_normalizado := ctrx_quarter * 1.2 ]
  
  
  
  #variable extraida de una tesis de maestria de Irlanda
  dataset[  , mpayroll_sobre_edad  := mpayroll / cliente_edad ]
  
  #se crean los nuevos campos para MasterCard  y Visa, teniendo en cuenta los NA's
  #varias formas de combinar Visa_status y Master_status
  dataset[ , mv_status01       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
  dataset[ , mv_status02       := Master_status +  Visa_status ]
  dataset[ , mv_status03       := pmax( ifelse( is.na(Master_status), 10, Master_status) , ifelse( is.na(Visa_status), 10, Visa_status) ) ]
  dataset[ , mv_status04       := ifelse( is.na(Master_status), 10, Master_status)  +  ifelse( is.na(Visa_status), 10, Visa_status)  ]
  dataset[ , mv_status05       := ifelse( is.na(Master_status), 10, Master_status)  +  100*ifelse( is.na(Visa_status), 10, Visa_status)  ]
  
  dataset[ , mv_status06       := ifelse( is.na(Visa_status), 
                                          ifelse( is.na(Master_status), 10, Master_status), 
                                          Visa_status)  ]
  
  dataset[ , mv_status07       := ifelse( is.na(Master_status), 
                                          ifelse( is.na(Visa_status), 10, Visa_status), 
                                          Master_status)  ]
  
  
  #combino MasterCard y Visa
  dataset[ , mv_mfinanciacion_limite := rowSums( cbind( Master_mfinanciacion_limite,  Visa_mfinanciacion_limite) , na.rm=TRUE ) ]
  
  dataset[ , mv_Fvencimiento         := pmin( Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE) ]
  dataset[ , mv_Finiciomora          := pmin( Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE) ]
  dataset[ , mv_msaldototal          := rowSums( cbind( Master_msaldototal,  Visa_msaldototal) , na.rm=TRUE ) ]
  dataset[ , mv_msaldopesos          := rowSums( cbind( Master_msaldopesos,  Visa_msaldopesos) , na.rm=TRUE ) ]
  dataset[ , mv_msaldodolares        := rowSums( cbind( Master_msaldodolares,  Visa_msaldodolares) , na.rm=TRUE ) ]
  dataset[ , mv_mconsumospesos       := rowSums( cbind( Master_mconsumospesos,  Visa_mconsumospesos) , na.rm=TRUE ) ]
  dataset[ , mv_mconsumosdolares     := rowSums( cbind( Master_mconsumosdolares,  Visa_mconsumosdolares) , na.rm=TRUE ) ]
  dataset[ , mv_mlimitecompra        := rowSums( cbind( Master_mlimitecompra,  Visa_mlimitecompra) , na.rm=TRUE ) ]
  dataset[ , mv_madelantopesos       := rowSums( cbind( Master_madelantopesos,  Visa_madelantopesos) , na.rm=TRUE ) ]
  dataset[ , mv_madelantodolares     := rowSums( cbind( Master_madelantodolares,  Visa_madelantodolares) , na.rm=TRUE ) ]
  dataset[ , mv_fultimo_cierre       := pmax( Master_fultimo_cierre, Visa_fultimo_cierre, na.rm = TRUE) ]
  dataset[ , mv_mpagado              := rowSums( cbind( Master_mpagado,  Visa_mpagado) , na.rm=TRUE ) ]
  dataset[ , mv_mpagospesos          := rowSums( cbind( Master_mpagospesos,  Visa_mpagospesos) , na.rm=TRUE ) ]
  dataset[ , mv_mpagosdolares        := rowSums( cbind( Master_mpagosdolares,  Visa_mpagosdolares) , na.rm=TRUE ) ]
  dataset[ , mv_fechaalta            := pmax( Master_fechaalta, Visa_fechaalta, na.rm = TRUE) ]
  dataset[ , mv_mconsumototal        := rowSums( cbind( Master_mconsumototal,  Visa_mconsumototal) , na.rm=TRUE ) ]
  dataset[ , mv_cconsumos            := rowSums( cbind( Master_cconsumos,  Visa_cconsumos) , na.rm=TRUE ) ]
  dataset[ , mv_cadelantosefectivo   := rowSums( cbind( Master_cadelantosefectivo,  Visa_cadelantosefectivo) , na.rm=TRUE ) ]
  dataset[ , mv_mpagominimo          := rowSums( cbind( Master_mpagominimo,  Visa_mpagominimo) , na.rm=TRUE ) ]
  
  #a partir de aqui juego con la suma de Mastercard y Visa
  dataset[ , mvr_Master_mlimitecompra:= Master_mlimitecompra / mv_mlimitecompra ]
  dataset[ , mvr_Visa_mlimitecompra  := Visa_mlimitecompra / mv_mlimitecompra ]
  dataset[ , mvr_msaldototal         := mv_msaldototal / mv_mlimitecompra ]
  dataset[ , mvr_msaldopesos         := mv_msaldopesos / mv_mlimitecompra ]
  dataset[ , mvr_msaldopesos2        := mv_msaldopesos / mv_msaldototal ]
  dataset[ , mvr_msaldodolares       := mv_msaldodolares / mv_mlimitecompra ]
  dataset[ , mvr_msaldodolares2      := mv_msaldodolares / mv_msaldototal ]
  dataset[ , mvr_mconsumospesos      := mv_mconsumospesos / mv_mlimitecompra ]
  dataset[ , mvr_mconsumosdolares    := mv_mconsumosdolares / mv_mlimitecompra ]
  dataset[ , mvr_madelantopesos      := mv_madelantopesos / mv_mlimitecompra ]
  dataset[ , mvr_madelantodolares    := mv_madelantodolares / mv_mlimitecompra ]
  dataset[ , mvr_mpagado             := mv_mpagado / mv_mlimitecompra ]
  dataset[ , mvr_mpagospesos         := mv_mpagospesos / mv_mlimitecompra ]
  dataset[ , mvr_mpagosdolares       := mv_mpagosdolares / mv_mlimitecompra ]
  dataset[ , mvr_mconsumototal       := mv_mconsumototal  / mv_mlimitecompra ]
  dataset[ , mvr_mpagominimo         := mv_mpagominimo  / mv_mlimitecompra ]
  
  #Aqui debe usted agregar sus propias nuevas variables
  
  #valvula de seguridad para evitar valores infinitos
  #paso los infinitos a NULOS
  infinitos      <- lapply(names(dataset),function(.name) dataset[ , sum(is.infinite(get(.name)))])
  infinitos_qty  <- sum( unlist( infinitos) )
  if( infinitos_qty > 0 )
  {
    cat( "ATENCION, hay", infinitos_qty, "valores infinitos en tu dataset. Seran pasados a NA\n" )
    dataset[mapply(is.infinite, dataset)] <<- NA
  }
  
  
  #valvula de seguridad para evitar valores NaN  que es 0/0
  #paso los NaN a 0 , decision polemica si las hay
  #se invita a asignar un valor razonable segun la semantica del campo creado
  nans      <- lapply(names(dataset),function(.name) dataset[ , sum(is.nan(get(.name)))])
  nans_qty  <- sum( unlist( nans) )
  if( nans_qty > 0 )
  {
    cat( "ATENCION, hay", nans_qty, "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n" )
    cat( "Si no te gusta la decision, modifica a gusto el programa!\n\n")
    dataset[mapply(is.nan, dataset)] <<- 0
  }
  
}

#------------------------------------------------------------------------------
#se calculan para los 6 meses previos el minimo, maximo y tendencia calculada con cuadrados minimos
#la formula de calculo de la tendencia puede verse en https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
#para la maxíma velocidad esta funcion esta escrita en lenguaje C, y no en la porqueria de R o Python

cppFunction('NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde ) 
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;
  int n = pcolumna.size();
  NumericVector out( 5*n );
  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;
    int  libre    = 0 ;
    int  xvalor   = 1 ;
    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;
       if( !R_IsNA( a ) ) 
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }
       xvalor++ ;
    }
    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;
      for( int h=1; h<libre; h++)
      { 
        xsum  += x[h] ;
        ysum  += y[h] ; 
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;
        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }
      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ; 
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }
  return  out;
}')

#------------------------------------------------------------------------------
#calcula la tendencia de las variables cols de los ultimos 6 meses
#la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
#La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas  <- function( dataset, cols, ventana=6, tendencia=TRUE, minimo=TRUE, maximo=TRUE, promedio=TRUE, 
                                 ratioavg=FALSE, ratiomax=FALSE)
{
  gc()
  #Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion  <- ventana
  
  last  <- nrow( dataset )
  
  #creo el vector_desde que indica cada ventana
  #de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids   <- dataset$numero_de_cliente
  
  vector_desde  <- seq( -ventana_regresion+2,  nrow(dataset)-ventana_regresion+1 )
  vector_desde[ 1:ventana_regresion ]  <-  1
  
  for( i in 2:last )  if( vector_ids[ i-1 ] !=  vector_ids[ i ] ) {  vector_desde[i] <-  i }
  for( i in 2:last )  if( vector_desde[i] < vector_desde[i-1] )  {  vector_desde[i] <-  vector_desde[i-1] }
  
  for(  campo  in   cols )
  {
    nueva_col     <- fhistC( dataset[ , get(campo) ], vector_desde ) 
    
    if(tendencia)  dataset[ , paste0( campo, "_tend", ventana) := nueva_col[ (0*last +1):(1*last) ]  ]
    if(minimo)     dataset[ , paste0( campo, "_min", ventana)  := nueva_col[ (1*last +1):(2*last) ]  ]
    if(maximo)     dataset[ , paste0( campo, "_max", ventana)  := nueva_col[ (2*last +1):(3*last) ]  ]
    if(promedio)   dataset[ , paste0( campo, "_avg", ventana)  := nueva_col[ (3*last +1):(4*last) ]  ]
    if(ratioavg)   dataset[ , paste0( campo, "_ratioavg", ventana)  := get(campo) /nueva_col[ (3*last +1):(4*last) ]  ]
    if(ratiomax)   dataset[ , paste0( campo, "_ratiomax", ventana)  := get(campo) /nueva_col[ (2*last +1):(3*last) ]  ]
  }
  
}
#------------------------------------------------------------------------------
#agrega al dataset nuevas variables {0,1} que provienen de las hojas de un Random Forest

AgregaVarRandomForest  <- function( num.trees, max.depth, min.node.size, mtry)
{
  gc()
  dataset[ , clase01:= ifelse( clase_ternaria=="CONTINUA", 0, 1 ) ]
  
  campos_buenos  <- setdiff( colnames(dataset), c("clase_ternaria" ) )
  
  dataset_rf  <- copy( dataset[ , campos_buenos, with=FALSE] )
  azar  <- runif( nrow(dataset_rf) )
  dataset_rf[ , entrenamiento := as.integer( foto_mes>= 202009 &  foto_mes<= 202101 & ( clase01==1 | azar < 0.10 )) ]
  
  #imputo los nulos, ya que ranger no acepta nulos
  #Leo Breiman, ¿por que le temias a los nulos?
  dataset_rf  <- na.roughfix( dataset_rf )
  
  campos_buenos  <- setdiff( colnames(dataset_rf), c("clase_ternaria","entrenamiento" ) )
  modelo  <- ranger( formula= "clase01 ~ .",
                     data=  dataset_rf[ entrenamiento==1L, campos_buenos, with=FALSE  ] ,
                     classification= TRUE,
                     probability=   FALSE,
                     num.trees=     num.trees,
                     max.depth=     max.depth,
                     min.node.size= min.node.size,
                     mtry=          mtry
  )
  
  rfhojas  <- predict( object= modelo, 
                       data= dataset_rf[ , campos_buenos, with=FALSE ],
                       predict.all= TRUE,    #entrega la prediccion de cada arbol
                       type= "terminalNodes" #entrega el numero de NODO el arbol
  )
  
  for( arbol in 1:num.trees )
  {
    hojas_arbol  <- unique(  rfhojas$predictions[  , arbol  ] )
    
    for( pos in 1:length(hojas_arbol) )
    {
      nodo_id  <- hojas_arbol[ pos ]  #el numero de nodo de la hoja, estan salteados
      dataset[  ,  paste0( "rf_", sprintf( "%03d", arbol ), "_", sprintf( "%03d", nodo_id ) ) := 0L ]
      
      dataset[ which( rfhojas$predictions[ , arbol] == nodo_id ,  ), 
               paste0( "rf_", sprintf( "%03d", arbol ), "_", sprintf( "%03d", nodo_id ) ) := 1L ]
    }
  }
  
  rm( dataset_rf )
  dataset[ , clase01 := NULL ]
  
  gc()
}
#------------------------------------------------------------------------------
VPOS_CORTE  <- c()

fganancia_lgbm_meseta  <- function(probs, datos) 
{
  vlabels  <- get_field(datos, "label")
  vpesos   <- get_field(datos, "weight")
  
  tbl  <- as.data.table( list( "prob"=probs, "gan"= ifelse( vlabels==1 & vpesos > 1, 60000, -2000 ) ) )
  
  setorder( tbl, -prob )
  tbl[ , posicion := .I ]
  tbl[ , gan_acum :=  cumsum( gan ) ]
  setorder( tbl, -gan_acum )   #voy por la meseta
  
  gan  <- mean( tbl[ 1:500,  gan_acum] )  #meseta de tamaño 500
  
  pos_meseta  <- tbl[ 1:500,  median(posicion)]
  VPOS_CORTE  <<- c( VPOS_CORTE, pos_meseta )
  
  return( list( "name"= "ganancia", 
                "value"=  gan,
                "higher_better"= TRUE ) )
}
#------------------------------------------------------------------------------
#Elimina del dataset las variables que estan por debajo de la capa geologica de canaritos
#se llama varias veces, luego de agregar muchas variables nuevas, para ir reduciendo la cantidad de variables
# y así hacer lugar a nuevas variables importantes

GVEZ <- 1 

CanaritosAsesinos  <- function( canaritos_ratio=0.2 )
{
  gc()
  dataset[ , clase01:= ifelse( clase_ternaria=="CONTINUA", 0, 1 ) ]
  
  for( i  in 1:(ncol(dataset)*canaritos_ratio))  dataset[ , paste0("canarito", i ) :=  runif( nrow(dataset))]
  
  campos_buenos  <- setdiff( colnames(dataset), c("clase_ternaria","clase01", "foto_mes" ) )
  
  azar  <- runif( nrow(dataset) )
  dataset[ , entrenamiento := foto_mes>= 202008 &  foto_mes<= 202111  & ( clase01==1 | azar < 0.10 ) ]
  
  dtrain  <- lgb.Dataset( data=    data.matrix(  dataset[ entrenamiento==TRUE, campos_buenos, with=FALSE]),
                          label=   dataset[ entrenamiento==TRUE, clase01],
                          weight=  dataset[ entrenamiento==TRUE, ifelse(clase_ternaria=="BAJA+2", 1.0000001, 1.0)],
                          free_raw_data= FALSE
  )
  
  dvalid  <- lgb.Dataset( data=    data.matrix(  dataset[ foto_mes==202101, campos_buenos, with=FALSE]),
                          label=   dataset[ foto_mes==202101, clase01],
                          weight=  dataset[ foto_mes==202101, ifelse(clase_ternaria=="BAJA+2", 1.0000001, 1.0)],
                          free_raw_data= FALSE
  )
  
  
  param <- list( objective= "binary",
                 metric= "custom",
                 first_metric_only= TRUE,
                 boost_from_average= TRUE,
                 feature_pre_filter= FALSE,
                 verbosity= -100,
                 seed= 999983,
                 max_depth=  -1,         # -1 significa no limitar,  por ahora lo dejo fijo
                 min_gain_to_split= 0.0, #por ahora, lo dejo fijo
                 lambda_l1= 0.0,         #por ahora, lo dejo fijo
                 lambda_l2= 0.0,         #por ahora, lo dejo fijo
                 max_bin= 31,            #por ahora, lo dejo fijo
                 num_iterations= 9999,   #un numero muy grande, lo limita early_stopping_rounds
                 force_row_wise= TRUE,    #para que los alumnos no se atemoricen con tantos warning
                 learning_rate= 0.065, 
                 feature_fraction= 1.0,   #lo seteo en 1 para que las primeras variables del dataset no se vean opacadas
                 min_data_in_leaf= 260,
                 num_leaves= 60,
                 early_stopping_rounds= 200 )
  
  modelo  <- lgb.train( data= dtrain,
                        valids= list( valid= dvalid ),
                        eval= fganancia_lgbm_meseta,
                        param= param,
                        verbose= -100 )
  
  tb_importancia  <- lgb.importance( model= modelo )
  tb_importancia[  , pos := .I ]
  
  fwrite( tb_importancia, 
          file= paste0( "impo_", GVEZ ,".txt"),
          sep= "\t" )
  
  GVEZ  <<- GVEZ + 1
  
  umbral  <- tb_importancia[ Feature %like% "canarito", median(pos) + 2*sd(pos) ]  #Atencion corto en la mediana mas DOS desvios!!
  
  col_utiles  <- tb_importancia[ pos < umbral & !( Feature %like% "canarito"),  Feature ]
  col_utiles  <-  unique( c( col_utiles,  c("numero_de_cliente","foto_mes","clase_ternaria","mes") ) )
  col_inutiles  <- setdiff( colnames(dataset), col_utiles )
  
  dataset[  ,  (col_inutiles) := NULL ]
  
}
#------------------------------------------------------------------------------
#agrega para cada columna de cols una nueva variable _rank  que es un numero entre 0 y 1  del ranking de esa variable ese mes

Rankeador  <- function( cols )
{
  gc()
  sufijo  <- "_rank" 
  
  for( vcol in cols )
  {
    dataset[ , paste0( vcol, sufijo) := frank( get(vcol), ties.method= "random")/ .N, 
             by= foto_mes ]
  }
}
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------
#Aqui empieza el programa

setwd( "~/buckets/b1/" )

#cargo el dataset
dataset  <- fread( "./datasets/competencia1_historia_2022.csv.gz" )

#Elimino los campos problematicos
dataset[  , internet := NULL ]

#creo la carpeta donde va el experimento
# FE  representa  Feature Engineering
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/FE8150/", showWarnings = FALSE )
setwd("./exp/FE8150/")   #Establezco el Working Directory DEL EXPERIMENTO



#corrijo los  < foto_mes, campo >  que fueron pisados con cero
# Atencion, cambiar si se desea el metodo VelenPennini
Corregir_MachineLearning( dataset )  
# Corregir_VelenPennini( dataset )


#Agrego las variables manuales al dataset
AgregarVariables( dataset )


#--------------------------------------
#estas son las columnas a las que se puede agregar lags o media moviles ( todas menos las obvias )
cols_lagueables  <- copy(  setdiff( colnames(dataset), c("numero_de_cliente", "foto_mes", "clase_ternaria")  ) )

#ordeno el dataset por <numero_de_cliente, foto_mes> para poder hacer lags
#  es MUY  importante esta linea
setorder( dataset, numero_de_cliente, foto_mes )

#creo los campos lags de orden 1
dataset[ , paste0( cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"), 
         by= numero_de_cliente, 
         .SDcols= cols_lagueables ]

#agrego los delta lags
for( vcol in cols_lagueables )
{
  dataset[ , paste0(vcol, "_delta1") := get(vcol)  - get(paste0( vcol, "_lag1"))  ]
}

#creo los campos lags de orden 2
dataset[ , paste0( cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag2"), 
         by= numero_de_cliente, 
         .SDcols= cols_lagueables ]

#agrego los delta lags
for( vcol in cols_lagueables )
{
  dataset[ , paste0(vcol, "_delta2") := get(vcol)  - get(paste0( vcol, "_lag2"))  ]
}

#creo los campos lags de orden 3
dataset[ , paste0( cols_lagueables, "_lag3") := shift(.SD, 2, NA, "lag3"), 
         by= numero_de_cliente, 
         .SDcols= cols_lagueables ]

#agrego los delta lags
for( vcol in cols_lagueables )
{
  dataset[ , paste0(vcol, "_delta3") := get(vcol)  - get(paste0( vcol, "_lag3"))  ]
}
#--------------------------------------
#agrego las tendencias

TendenciaYmuchomas( dataset, 
                    cols= cols_lagueables,
                    ventana=   6,      # 6 meses de historia
                    tendencia= TRUE,
                    minimo=    FALSE,
                    maximo=    FALSE,
                    promedio=  TRUE,
                    ratioavg=  FALSE,
                    ratiomax=  FALSE  )


#------------------------------------------------------------------------------
#Agrego variables a partir de las hojas de un Random Forest

AgregaVarRandomForest( num.trees = 40,
                       max.depth = 5,
                       min.node.size = 500,
                       mtry = 15 )

gc()

#------------------------------------------------------------------------------
#Elimino las variables que no son tan importantes en el dataset

ncol( dataset )
CanaritosAsesinos( canaritos_ratio = 0.3 )
ncol( dataset )

#------------------------------------------------------------------------------
#grabo el dataset
fwrite( dataset,
        "dataset_7130lobfull.csv.gz",
        logical01= TRUE,
        sep= "," )

ME fui a dormir dejando cargando el FE con los ajustes por inflación. Veremos si finalizó

http://localhost:8888/edit/Desktop/itbamat2/datasets/exp_FE8150_dataset_7130lobfull.csv.gz

Finalizó muy bien y ahora voy a correr una BO 633 con ese dataset entrenando 6 meses - vamos a ver que nos da

In [ ]:
http://34.125.164.102/ 

kBO_iter  <- 100   #cantidad de iteraciones de la Optimizacion Bayesiana

# ATENCION  si NO se quiere utilizar  undersampling  se debe  usar  kundersampling <- 1.0
kundersampling  <- 1.0   # un undersampling de 0.1  toma solo el 10% de los CONTINUA

prob_min  <- 0.5/( 1 + kundersampling*39)
prob_max  <- pmin( 1.0, 4/( 1 + kundersampling*39) )

#Aqui se cargan los hiperparametros
hs <- makeParamSet( 
         makeNumericParam("learning_rate",    lower=  0.009   , upper=    0.3),
         makeNumericParam("feature_fraction", lower=  0.4    , upper=    1.0),
         makeIntegerParam("min_data_in_leaf", lower=  500      , upper= 3000),
         makeIntegerParam("num_leaves",       lower= 16L     , upper= 1024L),
         makeNumericParam("prob_corte",       lower= prob_min, upper= prob_max  )  #esto sera visto en clase en gran detalle
        )

kdataset       <-  "./exp/FE8150/dataset_7130lobfull.csv.gz"         # El dataset generado con Feature Engineering
ksemilla_azar  <- 777371  #Aqui poner la propia semilla
kexperimento   <- "HT6336mesesajustadofinalfe"
ktraining      <- c( 202101, 202012, 202011, 202009, 202008, 202007 )   #periodos en donde entreno

kPOS_ganancia  <- 78000
kNEG_ganancia  <- -2000


Al star corriendo la BO..veo un resultado con 104millones de ganancias... al ser 6 meses.. Estará bien? 

20220930 181340	binary	custom	TRUE	TRUE	FALSE	-100	-1	0	0	0	31	143	TRUE	777371	0.0733142521570204	0.4771509333211	1095	686	0.0438892233851948	104974000